# Yara Megaprimer
> What is Yara and how is it used

- toc: true 
- badges: true
- categories: [yara,tutorial]


## Overview

Yara rule is basically just a set of rules used to match some features in a file (binary). The Yara [scanning tool](https://github.com/virustotal/yara) can then take these rules and scan a set of files, identifying files that match the rule.

According to [wikipedia](https://en.wikipedia.org/wiki/YARA)
> It provides a rule-based approach to create descriptions of malware families based on textual or binary patterns. A description is essentially a YARA rule name, where these rules consist of sets of strings and a boolean expression.

### Rule Syntax Overview
An example rule from the [Yara GitHub](https://github.com/virustotal/yara) shows us that there are three main components to a rule; the `meta` section, the [`strings`](https://yara.readthedocs.io/en/stable/writingrules.html#strings) section, and the [`condition`](https://yara.readthedocs.io/en/stable/writingrules.html#conditions). 


```c
rule silent_banker : banker
{
    meta:
        description = "This is just an example"
        threat_level = 3
        in_the_wild = true

    strings:
        $a = {6A 40 68 00 30 00 00 6A 14 8D 91}
        $b = {8D 4D B0 2B C1 83 C0 27 99 6A 4E 59 F7 F9}
        $c = "UVODFRYSIHLNWPEJXQZAKCBGMT"

    condition:
        $a or $b or $c
}
```



### There Be Dragons!
![](https://i.imgur.com/6NumPKX.png)
> https://twitter.com/nunohaien/status/1480821031656898560

## Using Yara

### Rules for Identifying Malware (Threat Intel)
- Used to share intel about malware families 
- Usually tuned for unpacked malware (the actual malware code, not the packer)
- Needs a low false positive (FP) rate or it could end up being "bad intel"
- Usually used to automatically identify and classify malware samples (ex. malware tracker)

A good example of these "intel" rules is the public Yara repository [Malpedia](https://malpedia.caad.fkie.fraunhofer.de/)


### Rules Used Like AV Signatures
- Used to identify packed malware, and early-stage malware delivery chain artifacts (maldocs, packers, malicious js, etc.)
- Doesn't need to identify a specific malware family, just needs to identify "badness" or "evil"
- Usually used either in-line or for asyncronous enterprise scanning to detect malware
- FP rate needs to be low (SOC goes brrrrrrr)

A good example of these "AV" rules is the open source ["loki" scanner](https://github.com/Neo23x0/Loki) with the [signature-base ruleset](https://github.com/Neo23x0/signature-base) from [Florian Roth](https://twitter.com/cyb3rops) and its commercial counterpart THOR.


### Rules Used Exclusively For Hunting
- Used like a "loose" version of a AV signature, used for finding the needle in the haystack
- Good for identifying unknowns 
- Usually used for "hunting" either within an enterpise or in a malware repository
- Can have a high FP rate

A good example of these are the "feels bad" rules from [100 Days of Yara](https://github.com/100DaysofYARA/)


## Testing

- Packer detection
  [pkr_ce1a](https://malwarology.substack.com/p/malicious-packer-pkr_ce1a)

- Detects "bad"
  [some_packer](https://pastebin.com/Yq0SRAN9)

- .NET packer
  [msil_suspicious_use_of_strreverse.yar](https://github.com/dr4k0nia/yara-rules/blob/main/dotnet/msil_suspicious_use_of_strreverse.yar)

- lol PDB paths


```c
import "pe"

rule lol_pdb {
    meta:
        author = "BitsOfBinary"
        description = "Detects files that identify themselves as malware"
        reference = "https://bitsofbinary.github.io/yara/2023/01/04/100daysofyara-day-4.html"
        version = "1.0"
        date = "2023-01-04"
        DaysofYARA = "4/100"
        
    condition:
        pe.pdb_path icontains "locker" or
        pe.pdb_path icontains "stealer" or 
        pe.pdb_path icontains "crypter" or
        pe.pdb_path icontains "ransomware" or
        pe.pdb_path icontains "keylogger" or  
        pe.pdb_path icontains "exploit" 
        
}
```


## Questions

Why would we need to define a string with zerors before and after such as the following: "\x00Fluffy\x00"

- this defines the surrounding bytes for a string, instead of just matching the string with no context... in this particular case it's very inefficient since the null bytes don't add much context and will infact slow down the matching buy adding some generic bytes to the string, a fullword keyword and a string would be a better choice

Advanced regex yara rules examples. 

- no

Create yara rule for .NET using IL where the instructions may change between compiled samples and thus wildcarding is required. 

- Maybe Drakonia's rule? TBD...

How to utilise fuzzy hashing (SSDEEP, SDHASH, mvHASH-B) and fuzzy rules.

- in addition to the basic string pattern matching Yara supports some advanced features such as [hash calculation](https://yara.readthedocs.io/en/stable/modules/hash.html) using the hash module but this doesn't support fuzzy hashing... generally hash matching is very brittel and sort of defeats the purpose of yara but it can be useful in some contexts... for fuzzy hashing I'm not sure if this is available outside of a custom module?

When should we write a yara rule vs when to write a [CAPE rule](https://capev2.readthedocs.io/en/latest/customization/signatures.html)

- if you want the rule to be useable outside of CAPE then Yara is the best, one of the main advantages of Yara is its portablilty (you can share them! and they just work!)
- if you are only running your rule on CAPE and you want some more advanced custom features you may be enticed to use a CAPE specific rule

What are the advantages of YARA-L 2.0 vs the normal yara, and what use cases of YARA-L 2.0 rules to use? 

- we can probably ignore this unless it catches on widely, all of these "custom" yara versions that are not portable are not worth investigating unless you are using the platform they are designed for

## Custom Yara Engines
Avast experimented with compiled yara rules using their [YaraNG](https://github.com/avast/yarang) engine. Appearently with some decent improvemenst (35%) as detailed in their blog [YaraNG: Reinventing the YARA Scanner](https://engineering.avast.io/yarang-reinventing-the-yara-scanner/). They have have a [yara linter](https://github.com/avast/yls).


There is also a project underway to re-write the Yara engine in rust [yara-x](https://github.com/VirusTotal/yara-x/).


## Some Fun Hunting PDB Paths

- `D:\Mktmp\Amadey\StealerDLL\x64\Release\STEALERDLL.pdb`
- `o:\Programmieren\Codesoft Releases\_NEW BETATEST\Codesoft PW Stealer\v0.35_older\abcdef Version\release\Codesoft-PW_Stealer_Server.pdb`
- `C:\Users\Ledzz\Desktop\LedzzsHUBBBGOLD\LedzzsHUBBBGOLD\Nitro-Ransomware-master\NitroRansomware\obj\Debug\NitroRansomware.pdb`
- `c:\slam_ransomware_builder\ConsoleApp2\ConsoleApp2\obj\Debug\ConsoleApp2.pdb`
- `d:\ronin\PROJEKTY\Keylogger\src\Keylogger_v0.05\Release\Keylogger.pdb`
- `C:\Users\user\Desktop\ExploitKITS\C#\Windows\Trojans\CyberRAT\Client\obj\Debug\Client.pdb`
- `c:\Users\User\Desktop\5.2\Decrypter\decrypter\obj\Debug\Chaos ransomware decrypter.pdb`
- `C:\Users\ygzat\source\repos\Crypter Test2\Crypter\Crypter\obj\Debug\Crypter.pdb`
- `D:\scm\Italy\Ransomware\lkh\FileDecrytptor2\Build\Win32\Release\FileCrytptor.pdb`
- `D:\Mktmp\Amadey\StealerDLL\x64\Release\STEALERDLL.pdb`

